In [1]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from altair import Chart
import altair as alt

In [2]:
data_dir = Path(".").absolute().parent / "data"
model_dir = data_dir / "models"

df = pd.read_csv(data_dir / "single_unit_dataset_tidied.csv")
X = df[["firing_rate", "cv-isi"]].values
le = joblib.load(model_dir / "label_encoder.gz")
svm = joblib.load(model_dir / "cluster_svm.gz")
scaler = joblib.load(model_dir / "clustering_scaler.gz")

In [3]:
X = df[["firing_rate", "cv-isi"]].values
X = scaler.transform(X)
labels = svm.predict(X)
y_hat = le.inverse_transform(labels)
df2 = (
    pd.DataFrame(scaler.inverse_transform(X), columns=["Mean Firing Rate", "CV(ISI)"])
    .assign(cluster=y_hat)
)

In [5]:
Chart(df2).mark_point().encode(x="CV(ISI)", y="Mean Firing Rate", color="cluster")

alt.Chart(...)

In [6]:
df2.groupby("cluster")["Mean Firing Rate"].apply(np.mean)

cluster
fast_firing       9.065424
slow_irregular    1.159405
slow_regular      2.385719
Name: Mean Firing Rate, dtype: float64

In [7]:
df2.to_csv(data_dir /"single_unit_labelled.csv", index=False)

In [8]:
X.shape

(132, 2)

In [9]:
df["experiment"].nunique()

12